In [15]:
dic = {'name': 'Ali Khan', 'age': 23}

In [16]:
from box import ConfigBox
new_dic=ConfigBox(dic)

In [17]:
new_dic.name

'Ali Khan'

In [18]:
import os

In [19]:
os.getcwd()

'c:\\py.aman\\Data_Science-Projects\\Data_Science-Project'

In [7]:
os.chdir('..')

In [20]:
os.getcwd()

'c:\\py.aman\\Data_Science-Projects\\Data_Science-Project'

In [21]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [33]:
from src.Data_Science import *
from src.Data_Science.constants import *
from src.Data_Science.utils.common import read_yaml, create_directories
import os
import zipfile
import urllib.request as request

In [34]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=  CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config

In [35]:
## component-Data Ingestion

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [36]:

try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-02-25 18:17:38,939: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-25 18:17:38,942: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-25 18:17:38,947: INFO: common: yaml file: schema.yaml loaded successfully]
[2026-02-25 18:17:38,949: INFO: common: created directory at: artifacts]
[2026-02-25 18:17:38,951: INFO: common: created directory at: artifacts/data_ingestion]
[2026-02-25 18:17:38,953: INFO: 535005154: File already exists]


In [37]:
data_ingestion_config

DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://github.com/krishnaik06/datasets/raw/refs/heads/main/winequality-data.zip', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')